In [ ]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import StopLoss, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 1
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC])
reward_scheme = RiskAdjustedReturns(return_algorithm="sortino")

csv_file = tf.keras.utils.get_file(
    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN, randomize_time_slices=True)

wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallet_tuples=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

strategy.run(steps=10000)

W1212 19:10:29.347062 4352769472 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/module.py:31: The name tf.enable_resource_variables is deprecated. Please use tf.compat.v1.enable_resource_variables instead.

W1212 19:10:29.379207 4352769472 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:46: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W1212 19:10:29.380007 4352769472 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:47: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W1212 19:10:29.380762 4352769472 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:48: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train

Observation Data:
['open' 'high' 'low' 'close' 'volume' 'volume usd' 'USD' 'BTC'
 'USD_pending' 'BTC_pending']


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
I1212 19:11:06.782850 4352769472 trading_environment.py:358] Order: <Order: id=8bcd59b3-9103-4250-84fd-300e67dddd18, status=OrderStatus.FILLED, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=1000.00 USD, siz

Execute:  <Order: id=8bcd59b3-9103-4250-84fd-300e67dddd18, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=1000.00 USD, size=1000.0, price=11622.61, criteria=None, path_id=8bcd59b3-9103-4250-84fd-300e67dddd18 | Recipes: <Recipe: id=c624eb4e-e4e3-4938-8d9b-b7fcc9084f9a, type=TradeType.MARKET, pair=USD/BTC, criteria=StopLoss: StopDirection.EITHER | 0.03 (T/P) | 0.02 (S/L)>>
Fill:  <Trade: id=c2c44240-6635-4829-94ee-b39928e0fa26, order_id=8bcd59b3-9103-4250-84fd-300e67dddd18, step=1, base_symbol=USD, quote_symbol=BTC, side=TradeSide.BUY, type=TradeType.MARKET, quantity=997.00 USD, price=11622.61, commission=3.00 USD>
Total traded:  1000.0
Completed:  8bcd59b3-9103-4250-84fd-300e67dddd18 <Order: id=0e280017-eb77-45cb-837e-ca464979e5fe, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.SELL, pair=USD/BTC, quantity=0.08578108 BTC, size=997.0, price=11622.61, criteria=StopLoss: StopDirection.EITHER | 0.03 (T/P) | 0.02 (S/L), path_i

I1212 19:11:07.013632 4352769472 trading_environment.py:358] Order: <Order: id=a300432a-ad20-4ee7-bf48-b15442352be4, status=OrderStatus.FILLED, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=535.15 USD, size=535.15, price=11650.91, criteria=None, path_id=a300432a-ad20-4ee7-bf48-b15442352be4 | Recipes: >
I1212 19:11:07.014863 4352769472 trading_environment.py:359] Observation: [7.29000000e-05 7.30837000e-05 7.28999000e-05 7.30538000e-05
 1.57980000e-06 1.15310932e-02 3.56770000e-06 2.57270000e-12
 1.00000000e-16 8.33520630e-09]
I1212 19:11:07.016299 4352769472 trading_environment.py:360] P/L: 1.00710410398708
I1212 19:11:07.017093 4352769472 trading_environment.py:361] Reward (7): 0.16979104662551256
I1212 19:11:07.020607 4352769472 trading_environment.py:362] Performance:         BTC  BTC_pending     USD  USD_pending    net_worth step
0  0.000257     0.833521  356.77          0.0  10071.04104    7
I1212 19:11:07.052879 4352769472 trading_environment.py:358] Order: <O

...

Execute:  <Order: id=d2045935-1bb9-435e-a125-91faf810c42e, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=189.56 USD, size=189.56, price=10590.59, criteria=None, path_id=d2045935-1bb9-435e-a125-91faf810c42e | Recipes: <Recipe: id=66a8b409-72ef-4e58-8e2f-4700b13485c0, type=TradeType.MARKET, pair=USD/BTC, criteria=StopLoss: StopDirection.EITHER | 0.03 (T/P) | 0.06 (S/L)>>
Fill:  <Trade: id=5ec73a00-62d8-4dd5-8018-c7d93e6abee2, order_id=d2045935-1bb9-435e-a125-91faf810c42e, step=109, base_symbol=USD, quote_symbol=BTC, side=TradeSide.BUY, type=TradeType.MARKET, quantity=188.99 USD, price=10590.59, commission=0.57 USD>
Total traded:  189.56
Completed:  d2045935-1bb9-435e-a125-91faf810c42e <Order: id=e46ff5ba-d764-4798-9beb-4f331746d8e9, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.SELL, pair=USD/BTC, quantity=0.01784509 BTC, size=188.99, price=10590.59, criteria=StopLoss: StopDirection.EITHER | 0.03 (T/P) | 0.06 (S/L), path

I1212 19:11:11.700049 4352769472 trading_environment.py:358] Order: None
I1212 19:11:11.702862 4352769472 trading_environment.py:359] Observation: [7.56700000e-05 7.67750000e-05 7.55941000e-05 7.61505000e-05
 6.78240000e-06 5.15684975e-02 1.00000000e-16 1.26127100e-10
 1.00000000e-16 8.87689010e-09]
I1212 19:11:11.705682 4352769472 trading_environment.py:360] P/L: 0.9466681378517
I1212 19:11:11.706269 4352769472 trading_environment.py:361] Reward (115): -0.07006373010664228
I1212 19:11:11.710924 4352769472 trading_environment.py:362] Performance:         BTC  BTC_pending  USD  USD_pending    net_worth step
0  0.012613     0.887689  0.0          0.0  9466.681379  115
I1212 19:11:11.757199 4352769472 trading_environment.py:358] Order: None
I1212 19:11:11.759004 4352769472 trading_environment.py:359] Observation: [7.57579000e-05 7.59000000e-05 7.54245000e-05 7.56700000e-05
 3.62850000e-06 2.74351862e-02 1.00000000e-16 1.26127100e-10
 1.00000000e-16 8.87689010e-09]
I1212 19:11:11.761434 43

Execute:  <Order: id=ad9f10d6-87f7-49f8-96df-f32eb9eecae2, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.SELL, pair=USD/BTC, quantity=0.00000648 BTC, size=0.07, price=10810.0, criteria=StopLoss: StopDirection.EITHER | 0.01 (T/P) | 0.04 (S/L), path_id=63ca4af9-8039-4b4a-b449-52ad14b1b0d3 | Recipes: >
Fill:  <Trade: id=a8dcfa49-af6c-4e2c-99b0-e5f81bbafa81, order_id=ad9f10d6-87f7-49f8-96df-f32eb9eecae2, step=119, base_symbol=USD, quote_symbol=BTC, side=TradeSide.SELL, type=TradeType.MARKET, quantity=0.07 USD, price=10384.01, commission=0.00 USD>
Total traded:  0.07
Completed:  ad9f10d6-87f7-49f8-96df-f32eb9eecae2 None
Execute:  <Order: id=cd3439e7-def5-4189-a780-be5be9e65ddf, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.SELL, pair=USD/BTC, quantity=0.00113058 BTC, size=12.42, price=10985.56, criteria=StopLoss: StopDirection.EITHER | 0.01 (T/P) | 0.06 (S/L), path_id=55a5befa-aecb-44ab-8cc4-1c0027ffb086 | Recipes: >
Fill:  <Trade: id=d3c546e8-8aab-45

I1212 19:11:12.152745 4352769472 trading_environment.py:360] P/L: 0.93690780177614
I1212 19:11:12.153558 4352769472 trading_environment.py:361] Reward (124): -0.0782223407130442
I1212 19:11:12.156549 4352769472 trading_environment.py:362] Performance:         BTC  BTC_pending   USD  USD_pending    net_worth step
0  0.012616     0.887652  0.94          0.0  9369.078018  124
I1212 19:11:12.193339 4352769472 trading_environment.py:358] Order: <Order: id=4f7cbea1-1479-4bce-bec9-a0ebf28f3d46, status=OrderStatus.FILLED, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=0.94 USD, size=0.94, price=10396.15, criteria=None, path_id=4f7cbea1-1479-4bce-bec9-a0ebf28f3d46 | Recipes: >
I1212 19:11:12.194591 4352769472 trading_environment.py:359] Observation: [7.4269500e-05 7.4269500e-05 7.3903600e-05 7.4084700e-05 1.0914000e-06
 8.0790240e-03 1.0000000e-16 1.2616360e-10 1.0000000e-16 8.8774239e-09]
I1212 19:11:12.197806 4352769472 trading_environment.py:360] P/L: 0.9360264410889501
I1

Execute:  <Order: id=4f7cbea1-1479-4bce-bec9-a0ebf28f3d46, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=0.94 USD, size=0.94, price=10396.15, criteria=None, path_id=4f7cbea1-1479-4bce-bec9-a0ebf28f3d46 | Recipes: <Recipe: id=81ed4a38-a524-4464-880f-2135c2f7e952, type=TradeType.MARKET, pair=USD/BTC, criteria=StopLoss: StopDirection.EITHER | 0.01 (T/P) | 0.02 (S/L)>>
Fill:  <Trade: id=18b50d70-b209-4c1d-9876-b1bf4b192ff4, order_id=4f7cbea1-1479-4bce-bec9-a0ebf28f3d46, step=125, base_symbol=USD, quote_symbol=BTC, side=TradeSide.BUY, type=TradeType.MARKET, quantity=0.94 USD, price=10396.15, commission=0.00 USD>
Total traded:  0.94
Completed:  4f7cbea1-1479-4bce-bec9-a0ebf28f3d46 <Order: id=c8be7568-1661-4eaa-8759-c28ba7260fb3, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.SELL, pair=USD/BTC, quantity=0.00009042 BTC, size=0.94, price=10396.15, criteria=StopLoss: StopDirection.EITHER | 0.01 (T/P) | 0.02 (S/L), path_id=4f7cbe